In [ ]:
# 安裝必要套件
!pip install mtkresearch
!pip install vllm==0.4.3
!pip install parsedatetime
!pip install rapidfuzz #模糊搜尋套件

In [ ]:
import logging
from vllm import LLM, SamplingParams

# 初始化 LLM 模型
llm = LLM(
    model='MediaTek-Research/Breeze-7B-FC-v1_0',  # 確認此模型名稱是否正確
    tensor_parallel_size=1,  # Colab 通常只有一張 GPU
    gpu_memory_utilization=0.9,  # GPU 記憶體使用率上限
    dtype='half'  # 使用半精度浮點數運算以節省 GPU 記憶體
)

# 設定推論參數
turn_end_token_id = 61876  # <|im_end|>
params = SamplingParams(
    temperature=0.3,  # 提高溫度，增加生成文本的多樣性
    top_p=0.9,        # 提高 top_p，擴大生成範圍
    max_tokens=2048,  # 根據需要調整
    repetition_penalty=1.1,
    stop_token_ids=[turn_end_token_id]
)

# 定義一個簡單的推論函式
def _inference(prompt, llm, params):
    logging.info(f"Prompt sent to model: {prompt}")
    response = llm.generate(prompt, params)[0].outputs[0].text
    logging.info(f"Model response: {response}")
    return response


In [20]:
import pandas as pd
from datetime import datetime, timedelta
import pytz
import difflib
import logging
import re
import json

from rapidfuzz import process, fuzz

class CustomerService:
    def __init__(self, llm, functions):
        # 設定日誌
        logging.basicConfig(level=logging.INFO)

        # LLM 和函式描述
        self.llm = llm
        self.functions = functions

        # 從 Google 試算表讀取課表資料
        self.course_sheet_id = ""  # 更新為你的課程試算表 ID
        try:
            self.course_data = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{self.course_sheet_id}/export?format=csv")
            logging.info("成功讀取課程資料。")
        except Exception as e:
            logging.error(f"無法讀取課程試算表: {e}")
            self.course_data = pd.DataFrame()  # 初始化為空的 DataFrame

        # 從 Google 試算表讀取合作夥伴資料
        self.partner_sheet_id = ""  # 更新為你的合作夥伴試算表 ID
        try:
            self.partner_data = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{self.partner_sheet_id}/export?format=csv")
            logging.info("成功讀取合作夥伴資料。")
        except Exception as e:
            logging.error(f"無法讀取合作夥伴試算表: {e}")
            self.partner_data = pd.DataFrame()  # 初始化為空的 DataFrame

    def parse_chinese_date(self, date_string: str) -> list:
        """
        統一解析中文日期描述（相對 & 絕對 & 區間），將其轉為 yyyy-mm-dd 的列表。
        若無法解析，回傳空串列。
        """
        tz = pytz.timezone('Asia/Taipei')
        now = datetime.now(tz)
        date_string = date_string.strip()

        # 先嘗試處理「區間」描述 (例如 "2023年6月到8月", "2023年-2024年")
        range_pattern = re.compile(r'^(?P<start>.+?)(至|到|-)(?P<end>.+)$')
        range_match = range_pattern.match(date_string)
        if range_match:
            start_part = range_match.group('start').strip()
            end_part = range_match.group('end').strip()
            
            # 分別取得起始日期清單與結束日期清單
            start_dates = self._parse_single_date_or_period(start_part, now, tz)
            end_dates = self._parse_single_date_or_period(end_part, now, tz)

            if start_dates and end_dates:
                # 取清單的最小日期 & 最大日期，組成區間
                start_dt = datetime.strptime(min(start_dates), "%Y-%m-%d").replace(tzinfo=tz)
                end_dt = datetime.strptime(max(end_dates), "%Y-%m-%d").replace(tzinfo=tz)
                if start_dt > end_dt:
                    # 若使用者寫法怪怪的，則交換一下
                    start_dt, end_dt = end_dt, start_dt

                parsed_range = []
                current = start_dt
                while current <= end_dt:
                    parsed_range.append(current.strftime("%Y-%m-%d"))
                    current += timedelta(days=1)
                return parsed_range
            else:
                # 如果無法解析區間的前後，則直接回傳空列表
                return []

        # 若非區間格式，直接用單次解析
        return self._parse_single_date_or_period(date_string, now, tz)

    def _parse_single_date_or_period(self, date_string: str, now, tz):
        """
        用於解析「單一描述」(相對日期、或絕對日期： yyyy年、yyyy年mm月、yyyy年mm月dd日)。
        若為「2023年9月」會回傳該月份所有天數；若為「2023年」，會回傳該年所有天數；
        相對日期如「今天」「下週」「上個月」等也會處理。
        """
        date_string = date_string.strip()

        # 結果容器
        parsed_dates = []

        # 1.嘗試完整年-月-日 (e.g., "2023年9月20日")
        pattern_ymd = re.compile(r'^(?P<year>\d{4})年(?P<month>\d{1,2})月(?P<day>\d{1,2})日?$')
        match_ymd = pattern_ymd.match(date_string)
        if match_ymd:
            year = int(match_ymd.group('year'))
            month = int(match_ymd.group('month'))
            day = int(match_ymd.group('day'))
            try:
                d = datetime(year=year, month=month, day=day, tzinfo=tz)
                return [d.strftime("%Y-%m-%d")]
            except ValueError:
                return []

        # 2.嘗試「yyyy年mm月」 => 該月全部日期
        pattern_ym = re.compile(r'^(?P<year>\d{4})年(?P<month>\d{1,2})月$')
        match_ym = pattern_ym.match(date_string)
        if match_ym:
            year = int(match_ym.group('year'))
            month = int(match_ym.group('month'))
            first_day = datetime(year=year, month=month, day=1, tzinfo=tz)
            
            # 下個月1號 - 1天 => 該月最後一天
            if month == 12:
                next_month = datetime(year=year + 1, month=1, day=1, tzinfo=tz)
            else:
                next_month = datetime(year=year, month=month + 1, day=1, tzinfo=tz)
            last_day = next_month - timedelta(days=1)

            current = first_day
            while current <= last_day:
                parsed_dates.append(current.strftime("%Y-%m-%d"))
                current += timedelta(days=1)
            return parsed_dates

        # 3.嘗試「yyyy年」 => 該年全部日期
        pattern_y = re.compile(r'^(?P<year>\d{4})年$')
        match_y = pattern_y.match(date_string)
        if match_y:
            year = int(match_y.group('year'))
            first_day = datetime(year=year, month=1, day=1, tzinfo=tz)
            last_day = datetime(year=year, month=12, day=31, tzinfo=tz)
            current = first_day
            while current <= last_day:
                parsed_dates.append(current.strftime("%Y-%m-%d"))
                current += timedelta(days=1)
            return parsed_dates

        # 4.嘗試「相對日期」
        # EX.昨天, 今天, 明天
        
        relative_map = {
            r'^昨天$': -1,
            r'^今天$': 0,
            r'^明天$': 1
        }
        for pat, offset in relative_map.items():
            if re.match(pat, date_string):
                day = now + timedelta(days=offset)
                return [day.strftime("%Y-%m-%d")]

        # EX.上週 / 本週 / 下週 (假設「週一」為一週的開始)
        # weekday() => Monday=0, Sunday=6
        
        if re.match(r'^上(?:個)?週$|^上(?:個)?星期$', date_string):
            # 上週一
            start_of_this_week = now - timedelta(days=now.weekday())  # 本週一
            start_of_last_week = start_of_this_week - timedelta(days=7)
            for i in range(7):
                day = (start_of_last_week + timedelta(days=i))
                parsed_dates.append(day.strftime("%Y-%m-%d"))
            return parsed_dates

        if re.match(r'^本(?:個)?週$|^本(?:個)?星期$', date_string):
            start_of_this_week = now - timedelta(days=now.weekday())
            for i in range(7):
                day = (start_of_this_week + timedelta(days=i))
                parsed_dates.append(day.strftime("%Y-%m-%d"))
            return parsed_dates

        if re.match(r'^下(?:個)?週$|^下(?:個)?星期$', date_string):
            start_of_next_week = (now - timedelta(days=now.weekday())) + timedelta(days=7)
            for i in range(7):
                day = start_of_next_week + timedelta(days=i)
                parsed_dates.append(day.strftime("%Y-%m-%d"))
            return parsed_dates

        # EX.上個月 / 本月 / 下個月
        if re.match(r'^上(?:個)?月$', date_string):
            first_day_this_month = now.replace(day=1)
            last_day_last_month = first_day_this_month - timedelta(days=1)
            first_day_last_month = last_day_last_month.replace(day=1)
            current = first_day_last_month
            while current <= last_day_last_month:
                parsed_dates.append(current.strftime("%Y-%m-%d"))
                current += timedelta(days=1)
            return parsed_dates

        if re.match(r'^本(?:個)?月$', date_string):
            first_day_this_month = now.replace(day=1)
            # 下個月1號 - 1天
            next_month = first_day_this_month + timedelta(days=32)
            first_day_next_month = next_month.replace(day=1)
            last_day_this_month = first_day_next_month - timedelta(days=1)
            current = first_day_this_month
            while current <= last_day_this_month:
                parsed_dates.append(current.strftime("%Y-%m-%d"))
                current += timedelta(days=1)
            return parsed_dates

        if re.match(r'^下(?:個)?月$', date_string):
            first_day_next_month = (now.replace(day=1) + timedelta(days=32)).replace(day=1)
            next_next_month = first_day_next_month + timedelta(days=32)
            first_day_next_next_month = next_next_month.replace(day=1)
            last_day_next_month = first_day_next_next_month - timedelta(days=1)
            current = first_day_next_month
            while current <= last_day_next_month:
                parsed_dates.append(current.strftime("%Y-%m-%d"))
                current += timedelta(days=1)
            return parsed_dates

        # 5.如果仍無法解析 => 這裡可改成呼叫 LLM，但此處示範回傳空
        logging.warning(f"Date description '{date_string}' did not match any patterns,無法解析。")
        return []



    def get_date(self, date_string: str):
        """
        將中文日期（含相對/絕對/區間）描述轉換為具體的日期清單。
        """
        if not date_string:
            return []
        # 使用新的 parse_chinese_date 解析
        date_list = self.parse_chinese_date(date_string)
        if date_list:
            logging.info(f"'{date_string}' 解析成功，得到日期清單: {date_list}")
            return date_list
        else:
            logging.error(f"無法解析日期描述: {date_string}")
            return []


    # 課程資訊相關
    def get_course_topic(self, dates=[], relative_dates_description=""):
        return self._get_course_info('社課主題', dates, relative_dates_description)

    def get_course_location(self, dates=[], relative_dates_description=""):
        return self._get_course_info('地點', dates, relative_dates_description)

    def get_course_time(self, dates=[], relative_dates_description=""):
        return self._get_course_info('Time', dates, relative_dates_description)

    def get_course_type(self, dates=[], relative_dates_description=""):
        return self._get_course_info('社課種類', dates, relative_dates_description)

    def get_course_speaker(self, dates=[], relative_dates_description=""):
        return self._get_course_info('講者', dates, relative_dates_description)

    def get_course_outline(self, dates=[], relative_dates_description=""):
        return self._get_course_info('課程大綱', dates, relative_dates_description)

    def get_course_notes(self, dates=[], relative_dates_description=""):
        return self._get_course_info('備註', dates, relative_dates_description)

    def get_all_course_info(self, dates=[], relative_dates_description=""):
        """
        回傳指定日期或相對日期描述 (e.g. "2023年", "下個月", "2024年9月") 的所有課程資料。
        """
        if relative_dates_description:
            dates = self.get_date(relative_dates_description)
            logging.info(f"Parsed dates from relative description '{relative_dates_description}': {dates}")

        if not isinstance(dates, list):
            dates = [dates]
            logging.info(f"Single date converted to list: {dates}")

        results = []
        for date in dates:
            index_date = self.course_data["Date"] == date
            if index_date.any():
                course_info = self.course_data.loc[index_date].to_dict(orient='records')[0]
                results.append(course_info)
                logging.info(f"Found course info for date {date}: {course_info}")

        if not results:
            logging.warning(f"No course info found for dates: {dates}")
            return {'message': f"您指定的日期:{dates}，沒有課程。"}

        return results

    def _get_course_info(self, field_name, dates=[], relative_dates_description=""):
        """
        根據日期清單或相對日期描述來篩選課程資料，並返回指定欄位。
        """
        if relative_dates_description:
            dates = self.get_date(relative_dates_description)

        if not isinstance(dates, list):
            dates = [dates]

        result = []
        for date in dates:
            df_filtered = self.course_data[self.course_data["Date"] == date]
            if not df_filtered.empty:
                value = df_filtered[field_name].values[0]
                result.append({date: value})

        if not result:
            return {'message': f"在日期 {dates} 找不到課程，或欄位 {field_name} 無資料。"}

        return result

    def find_course_by_keyword(self, query, fields=["社課主題", "地點", "時間", "社課種類"], limit=10, threshold=60):
        logging.info(f"開始搜尋課程關鍵字: {query}，搜尋欄位: {fields}")
        results = []

        # 定義欄位的重要性順序
        field_priority = ["社課主題", "地點", "時間", "社課種類"]
        for field in field_priority:
            if field not in self.course_data.columns:
                logging.warning(f"欄位 '{field}' 不存在於課程資料中。")
                continue

            data_list = self.course_data[field].dropna().tolist()
            matches = process.extract(query.lower(), data_list, scorer=fuzz.WRatio, limit=limit)
            logging.info(f"欄位 '{field}' 找到的匹配結果: {matches}")

            filtered_matches = [match for match in matches if match[1] >= threshold]
            logging.info(f"欄位 '{field}' 過濾後的匹配結果（分數 >= {threshold}）: {filtered_matches}")

            for match in filtered_matches:
                matched_value = match[0]
                score = match[1]
                course_infos = self.course_data[self.course_data[field] == matched_value].to_dict(orient='records')
                for course_info in course_infos:
                    results.append({
                        'query': query,
                        'matched_field': field,
                        'matched_value': matched_value,
                        'score': score,
                        'course_info': course_info
                    })
                    logging.info(f"找到匹配課程: {matched_value} (分數: {score})")

        if not results:
            logging.warning(f"沒有找到與關鍵字 '{query}' 匹配的課程。")
            return {'result': f"沒有找到與 '{query}' 匹配的課程。"}

        logging.info(f"總共找到 {len(results)} 個匹配課程。")
        return results

    # 合作夥伴資訊相關
    def get_partner_info(self, partner_names=[], relative_dates_description=""):
        return self._get_partner_info('合作單位', partner_names, relative_dates_description)

    def get_partner_collaboration_dates(self, partner_names=[], relative_dates_description=""):
        return self._get_partner_info('合作日期', partner_names, relative_dates_description)

    def _get_partner_info(self, field_name, partner_names=[], relative_dates_description=""):
        """
        從 partner_data 中篩選出符合 partner_names 或相對日期描述的資料，回傳指定欄位。
        """
        if relative_dates_description:
            dates = self.get_date(relative_dates_description)
            logging.info(f"從相對日期描述 '{relative_dates_description}' 解析出的日期範圍: {dates}")
            filtered_data = self.partner_data[self.partner_data['合作日期'].isin(dates)]
        else:
            filtered_data = self.partner_data.copy()

        if partner_names:
            filtered_data = filtered_data[filtered_data['合作單位'].isin(partner_names)]
            logging.info(f"根據合作單位名稱 {partner_names} 過濾後的合作夥伴資料數量: {len(filtered_data)}")

        if filtered_data.empty:
            logging.warning(f"指定的條件沒有找到任何合作夥伴資訊。")
            return {'message': f"指定的條件沒有找到任何合作夥伴資訊。" }

        return filtered_data[field_name].dropna().unique().tolist()

    def get_event_name(self, event_dates=[], relative_dates_description=""):
        return self._get_partner_event_info('活動名稱', event_dates, relative_dates_description)

    def get_event_location(self, event_dates=[], relative_dates_description=""):
        return self._get_partner_event_info('活動地點', event_dates, relative_dates_description)

    def get_event_time(self, event_dates=[], relative_dates_description=""):
        return self._get_partner_event_info('活動時間', event_dates, relative_dates_description)

    def get_event_description(self, event_dates=[], relative_dates_description=""):
        return self._get_partner_event_info('活動簡述', event_dates, relative_dates_description)

    def _get_partner_event_info(self, field_name, event_dates=[], relative_dates_description=""):
        """
        根據日期或相對日期描述，返回 partner_data 裏該欄位的資訊。
        """
        if relative_dates_description:
            event_dates = self.get_date(relative_dates_description)
        if not isinstance(event_dates, list):
            event_dates = [event_dates]

        filtered_data = self.partner_data[self.partner_data['合作日期'].isin(event_dates)]
        if filtered_data.empty:
            logging.warning(f"指定的條件沒有找到任何合作夥伴資訊。")
            return {'message': f"指定的條件沒有找到任何活動資訊。" }

        return filtered_data[field_name].dropna().unique().tolist()

    def get_all_partner_info(self, partner_names=[], relative_dates_description=""):
        if relative_dates_description:
            dates = self.get_date(relative_dates_description)
            logging.info(f"從相對日期描述 '{relative_dates_description}' 解析出的日期範圍: {dates}")
            filtered_data = self.partner_data[self.partner_data['合作日期'].isin(dates)]
        else:
            filtered_data = self.partner_data.copy()

        if partner_names:
            filtered_data = filtered_data[filtered_data['合作單位'].isin(partner_names)]
            logging.info(f"根據合作單位名稱 {partner_names} 過濾後的合作夥伴資料數量: {len(filtered_data)}")

        if filtered_data.empty:
            logging.warning(f"指定的條件沒有找到任何合作夥伴資訊。")
            return {'message': f"指定的條件沒有找到任何合作夥伴資訊。"}

        return filtered_data.to_dict(orient='records')

    def find_partner_event_by_keyword(self, query, fields=["活動名稱", "活動地點", "活動時間", "活動簡述"], limit=10, threshold=60):
        """
        根據關鍵字（模糊）搜尋相關合作夥伴的活動。
        """
        logging.info(f"開始搜尋合作夥伴活動關鍵字: {query}，搜尋欄位: {fields}")

        results = []
        for field in fields:
            if field not in self.partner_data.columns:
                logging.warning(f"欄位 '{field}' 不存在於合作夥伴資料中。")
                continue

            data_list = self.partner_data[field].dropna().tolist()
            matches = process.extract(query, data_list, scorer=fuzz.partial_ratio, limit=limit)
            logging.info(f"欄位 '{field}' 找到的匹配結果: {matches}")

            filtered_matches = [match for match in matches if match[1] >= threshold]
            logging.info(f"欄位 '{field}' 過濾後的匹配結果（分數 >= {threshold}）: {filtered_matches}")

            for match in filtered_matches:
                matched_value = match[0]
                score = match[1]
                event_infos = self.partner_data[self.partner_data[field] == matched_value].to_dict(orient='records')
                for event_info in event_infos:
                    results.append({
                        'query': query,
                        'matched_field': field,
                        'matched_value': matched_value,
                        'score': score,
                        'event_info': event_info
                    })
                    logging.info(f"找到匹配活動: {matched_value} (分數: {score})")

        if not results:
            logging.warning(f"沒有找到與關鍵字 '{query}' 匹配的合作夥伴活動。")
            return {'result': f"沒有找到與 '{query}' 匹配的合作夥伴活動。"}

        logging.info(f"總共找到 {len(results)} 個匹配合作夥伴活動。")
        return results



In [21]:
import json

# Define callable function descriptions (OpenAI function calling format)
functions = [
    # Course Information Related Functions
    {
        "name": "get_course_topic",
        "description": "Retrieve course topics based on specific dates or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of dates in yyyy-mm-dd format."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression, which can be either relative (e.g. '上個月', '這週', '明天') or absolute (e.g. '2023年', '2024年9月', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_course_location",
        "description": "Retrieve course locations based on specific dates or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of dates in yyyy-mm-dd format."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression, which can be either relative (e.g. '上個月', '這週', '明天') or absolute (e.g. '2023年', '2024年9月', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_course_time",
        "description": "Retrieve course times based on specific dates or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of dates in yyyy-mm-dd format."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression, which can be either relative (e.g. '上個月', '這週', '明天') or absolute (e.g. '2023年', '2024年9月', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_course_type",
        "description": "Retrieve course types based on specific dates or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of dates in yyyy-mm-dd format."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression, which can be either relative (e.g. '上個月', '這週', '明天') or absolute (e.g. '2023年', '2024年9月', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_course_speaker",
        "description": "Retrieve course speakers based on specific dates or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of dates in yyyy-mm-dd format."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression, which can be either relative (e.g. '上個月', '這週', '明天') or absolute (e.g. '2023年', '2024年9月', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_course_outline",
        "description": "Retrieve course outlines based on specific dates or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of dates in yyyy-mm-dd format."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression, which can be either relative (e.g. '上個月', '這週', '明天') or absolute (e.g. '2023年', '2024年9月', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_course_notes",
        "description": "Retrieve course notes based on specific dates or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of dates in yyyy-mm-dd format."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression, which can be either relative (e.g. '上個月', '這週', '明天') or absolute (e.g. '2023年', '2024年9月', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_all_course_info",
        "description": "Retrieve all course information based on specific dates or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of dates in yyyy-mm-dd format."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression, which can be either relative (e.g. '上個月', '這週', '明天') or absolute (e.g. '2023年', '2024年9月', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "find_course_by_keyword",
        "description": "Find the closest matching course based on a course topic keyword (query).",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Course topic query string, can be partial keywords or fuzzy descriptions."
                }
            },
            "required": ["query"]
        }
    },

    # Partner Information Related Functions
    {
        "name": "get_partner_info",
        "description": "Retrieve partner information based on partner names or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "partner_names": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of partner names."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression for the partner's collaboration date, which can be either relative or absolute (e.g. '上個月', '2023年', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_partner_collaboration_dates",
        "description": "Retrieve collaboration dates based on partner names or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "partner_names": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of partner names."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression for the partner's collaboration date, which can be either relative or absolute (e.g. '上個月', '2023年', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_event_name",
        "description": "Retrieve event names based on event dates or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "event_dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of event dates in yyyy-mm-dd format."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression for the event date, which can be either relative or absolute (e.g. '上週', '2023年', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_event_location",
        "description": "Retrieve event locations based on event dates or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "event_dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of event dates in yyyy-mm-dd format."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression for the event date, which can be either relative or absolute (e.g. '上週', '2023年', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_event_time",
        "description": "Retrieve event times based on event dates or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "event_dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of event dates in yyyy-mm-dd format."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression for the event date, which can be either relative or absolute (e.g. '上週', '2023年', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_event_description",
        "description": "Retrieve event descriptions based on event dates or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "event_dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of event dates in yyyy-mm-dd format."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression for the event date, which can be either relative or absolute (e.g. '上週', '2023年', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "get_all_partner_info",
        "description": "Retrieve all partner information based on partner names or Chinese date expressions.",
        "parameters": {
            "type": "object",
            "properties": {
                "partner_names": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of partner names."
                },
                "relative_dates_description": {
                    "type": "string",
                    "description": "A Chinese date expression for the partner's collaboration date, which can be either relative or absolute (e.g. '上個月', '2023年', '2023年6月到8月')."
                }
            },
            "required": []
        }
    },
    {
        "name": "find_partner_event_by_keyword",
        "description": "Find the closest matching partner event based on an event keyword (query).",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Event keyword query string, can be partial keywords or fuzzy descriptions."
                }
            },
            "required": ["query"]
        }
    }
]


In [22]:
# 初始化 CustomerService 實例
my_service = CustomerService(llm=llm, functions=functions)

In [23]:
# 將自訂函式名稱與 CustomerService 實例方法對應起來
mapping = {
    # 課程資訊相關
    'get_course_topic': my_service.get_course_topic,
    'get_course_location': my_service.get_course_location,
    'get_course_time': my_service.get_course_time,
    'get_course_type': my_service.get_course_type,
    'get_course_speaker': my_service.get_course_speaker,
    'get_course_outline': my_service.get_course_outline,
    'get_course_notes': my_service.get_course_notes,
    'get_all_course_info': my_service.get_all_course_info,
    'find_course_by_keyword': my_service.find_course_by_keyword,

    # 合作夥伴資訊相關
    'get_partner_info': my_service.get_partner_info,
    'get_partner_collaboration_dates': my_service.get_partner_collaboration_dates,
    'get_event_name': my_service.get_event_name,
    'get_event_location': my_service.get_event_location,
    'get_event_time': my_service.get_event_time,
    'get_event_description': my_service.get_event_description,
    'get_all_partner_info': my_service.get_all_partner_info,
    'find_partner_event_by_keyword': my_service.find_partner_event_by_keyword
}


In [43]:
import json
from mtkresearch.llm.prompt import MRPromptV2

# 初始化 MRPromptV2
prompt_engine = MRPromptV2()

# 建立初始對話歷史
conversations = [
    {"role": "system", "content": "You are a customer service staff of NTU AI club(台大人工智慧應用社). Members will ask you about club information or announcements."},
    {"role": "user", "content": "請問2024的課程有哪些?"}
    # {"role": "user", "content": "2023年有課程嗎?"}
    # {"role": "user", "content": "請問9月有什麼課程嗎?"}
    # {"role": "user", "content": "合作夥伴有誰?"}
    # {"role": "user", "content": "大型語言模型"}
    # {"role": "user", "content": "社課的地點都在哪裡?"}
    # {"role": "user", "content": "10月的社課的地點都在哪裡?"}
    # {"role": "user", "content": "請問9月有什麼課程嗎?"}
    # {"role": "user", "content": "上個月的合作夥伴有誰？"}
    # {"role": "user", "content": "請問2024年9月有什麼課程嗎?"}
    # {"role": "user", "content": "請問9月有什麼課程嗎?"}
    # {"role": "user", "content": "可以幫我整理課程資訊嗎?"}

]

# 取得 prompt 並進行推論
prompt = prompt_engine.get_prompt(conversations, functions=functions)
output_str = _inference(prompt, llm, params)
result = prompt_engine.parse_generated_str(output_str)

print("模型初次回應解析：", result)


# 1.如果有 tool_calls，就先將模型傳回的 arguments 修正後，再 append 到 conversations
if 'tool_calls' in result and result['tool_calls']:
    # 只示範處理第一個 tool_call
    tool_call = result['tool_calls'][0]
    func_name = tool_call['function']['name']
    try:
        arguments_dict = json.loads(tool_call['function']['arguments'])
        # 容錯：若 dates 是字串，就包成 list
        if "dates" in arguments_dict and isinstance(arguments_dict["dates"], str):
            arguments_dict["dates"] = [arguments_dict["dates"]]
        if "event_dates" in arguments_dict and isinstance(arguments_dict["event_dates"], str):
            arguments_dict["event_dates"] = [arguments_dict["event_dates"]]

        # 把修正後的 arguments 再轉回字串，存回 result
        tool_call['function']['arguments'] = json.dumps(arguments_dict, ensure_ascii=False)

    except json.JSONDecodeError:
        print("函式調用的參數無法解析為 JSON 格式。")

# 注意：此時 result['tool_calls'][0]['function']['arguments'] 已是合法的 array

# 將(已修正的) result 加回對話中
conversations.append(result)

# 2.執行函式
if 'tool_calls' in result and result['tool_calls']:
    tool_call = result['tool_calls'][0]
    func_name = tool_call['function']['name']
    func = mapping.get(func_name)

    if func:
        try:
        # 這裡再 parse 一次 (已修正後的) arguments
            arguments = json.loads(tool_call['function']['arguments'])
        except json.JSONDecodeError:
            print("函式調用的參數無法解析為 JSON 格式。")
            arguments = {}

        # 呼叫對應的 Python 函式
        try:
            called_result = func(**arguments)
        except Exception as e:
            print(f"呼叫函式 {func_name} 時發生錯誤: {e}")
            called_result = {"error": "處理您的請求時發生錯誤。"}

        # 在將函式呼叫結果加回對話前，也不用再修改什麼了
        tool_response = {
            'role': 'tool',
            'tool_call_id': tool_call.get('id', ''),
            'name': func_name,
            'content': json.dumps(called_result, ensure_ascii=False)
        }
        conversations.append(tool_response)

        # 再次產生新的 prompt 並進行推論，讓模型根據函式結果做進一步回應
        prompt = prompt_engine.get_prompt(conversations, functions=functions)
        output_str2 = _inference(prompt, llm, params)
        result2 = prompt_engine.parse_generated_str(output_str2)
        print(result2.get('content', ''))
    else:
        print(f"函式名稱 {func_name} 未在 mapping 中找到對應的方法。")
else:
    # 如果模型回應中不包含函式調用，直接輸出內容
    print(result.get('content', '無回應內容。'))


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s, Generation Speed: 53.57 toks/s]


模型初次回應解析： {'role': 'assistant', 'tool_calls': [{'id': 'call_KRPUPcCF4ErBoWYccU78VuyF', 'type': 'function', 'function': {'name': 'get_course_topic', 'arguments': '{"dates": ["2024-01-01"], "relative_dates_description": "2024年"}'}}]}


Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.84s/it, Generation Speed: 64.38 toks/s]

在2024年，NTU AI Club有許多精彩的課程和活動。以下是一些重點：

- **社團介紹大會**：2024年9月12日
- **機器學習導論**：2024年9月16日
- **Nvidia-[超越視覺與語言能力：多模態大型語言模型的進展]**：2024年9月23日
- **Introduction to generative AI Part 1**：2024年9月26日
- **滿拓科技-[生成式AI的地端應用]**：2024年9月30日
- **技術讀書會：技術交流與分享**：定期舉行
- **mentor hour: 討論主題、確認專案方向**：2024年10月7日
- **Introduction to generative AI Part 2**：2024年11月4日
- **機器學習導論 (深度學習)**：2024年11月7日
- **USPACE-[topic]**：2024年11月11日
- **強化式學習**：2024年11月14日
- **Pincollege-Creative AI**：2024年11月18日
- **The next AI frontier is agentic - A brief introduction to agentic AI systems**：2024年11月21日
- **Mindify**：2024年11月25日
- **台大鳥巢：科技創業**：2024年12月2日
- **mentor hour: 討論與諮詢**：2024年12月5日
- **專案期末成果發表會**：2024年12月23日

請注意，日期可能會有所變化，具體詳情請查詢我們的官方渠道或網站。




---
Colab執行後，串接到LINE平台上


In [ ]:
!pip install flask
!pip install line-bot-sdk
!pip install pyngrok

In [ ]:
# 初始化 Flask 應用程式
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

app = Flask(__name__)

# 替換為您的 Channel Access Token 和 Channel Secret
LINE_CHANNEL_ACCESS_TOKEN = 'YOUR_CHANNEL_ACCESS_TOKEN'
LINE_CHANNEL_SECRET = 'YOUR_CHANNEL_SECRET'

line_bot_api = LineBotApi(LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

@app.route("/callback", methods=['POST'])
def callback():
    # 獲取 X-Line-Signature header
    signature = request.headers['X-Line-Signature']

    # 獲取請求的 body
    body = request.get_data(as_text=True)
    app.logger.info(f"Request body: {body}")

    # 處理 webhook 事件
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [47]:
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_message = event.message.text
    logging.info(f"Received message from user: {user_message}")

    # 建立初始對話歷史
    conversations = [
        {
            "role": "system",
            "content": "You are a customer service staff of NTU AI club(台大人工智慧應用社). Members will ask you about club information or announcements."
        },
        {
            "role": "user",
            "content": user_message
        }
    ]

    # 第一次：取得 prompt 並進行推論
    prompt = prompt_engine.get_prompt(conversations, functions=functions)
    output_str = _inference(prompt, llm, params)
    result = prompt_engine.parse_generated_str(output_str)

    # 先看模型是否有 tool_calls
    if 'tool_calls' in result and result['tool_calls']:
        # 只示範第1個函式呼叫
        tool_call = result['tool_calls'][0]
        func_name = tool_call['function']['name']

        # 先檢查 arguments，如果 dates 是字串就轉成 list，避免 JSON Schema 驗證錯誤
        try:
            arguments_dict = json.loads(tool_call['function']['arguments'])

            # 若 dates 是字串，改成陣列
            if "dates" in arguments_dict and isinstance(arguments_dict["dates"], str):
                arguments_dict["dates"] = [arguments_dict["dates"]]
            # 若 event_dates 是字串，改成陣列
            if "event_dates" in arguments_dict and isinstance(arguments_dict["event_dates"], str):
                arguments_dict["event_dates"] = [arguments_dict["event_dates"]]

            # 修正後再存回 result
            tool_call['function']['arguments'] = json.dumps(arguments_dict, ensure_ascii=False)
        except json.JSONDecodeError:
            # 如果解析 JSON 失敗，直接回覆使用者
            response_message = "函式調用的參數無法解析為 JSON 格式。"
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text=response_message)
            )
            return

    # 到此，result 中的 tool_calls（若有）已被修正成符合 JSON schema
    # 接著再把 result append 進 conversations
    conversations.append(result)

    # 以下執行函式 (若存在)
    if 'tool_calls' in result and result['tool_calls']:
        tool_call = result['tool_calls'][0]
        func_name = tool_call['function']['name']
        func = mapping.get(func_name)
        if func:
            # 第二步：呼叫對應 Python 函式
            try:
                arguments = json.loads(tool_call['function']['arguments'])
            except json.JSONDecodeError:
                response_message = "函式調用的參數無法解析為 JSON 格式。"
                line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage(text=response_message)
                )
                return

            # 執行函式
            try:
                called_result = func(**arguments)
            except Exception as e:
                logging.error(f"呼叫函式 {func_name} 時發生錯誤: {e}")
                called_result = {"error": "處理您的請求時發生錯誤。"}

            # 把函式呼叫結果做成對話訊息
            tool_response = {
                'role': 'tool',
                'tool_call_id': tool_call.get('id', ''),
                'name': func_name,
                'content': json.dumps(called_result, ensure_ascii=False)
            }
            conversations.append(tool_response)

            # 第三步：再一次產生新的 prompt 並進行推論
            prompt = prompt_engine.get_prompt(conversations, functions=functions)
            output_str2 = _inference(prompt, llm, params)
            result2 = prompt_engine.parse_generated_str(output_str2)

            final_response = result2.get('content', '無回應內容。')
        else:
            final_response = f"函式名稱 {func_name} 未在 mapping 中找到對應的方法。"
    else:
        # 如果模型回應中不包含函式調用，直接輸出內容
        final_response = result.get('content', '無回應內容。')

    # 最後回應用戶
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=final_response)
    )


In [ ]:
# 啟動 Flask 應用程式
def run_app():
    app.run(host='0.0.0.0', port=0000)

# 設定並啟動 ngrok 隧道
from pyngrok import ngrok

# 設定 ngrok Authtoken
ngrok.set_auth_token("")  # ngrok Authtoken

# 開啟 ngrok 隧道
public_url = ngrok.connect(0000)
print(f"ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:0000\"")

# 在新線程中運行 Flask 應用程式
import threading
threading.Thread(target=run_app).start()